In this notebook we will implement a simplified version of the DCM model.

In [ ]:
import numpy
import os,sys
%matplotlib inline
import matplotlib.pyplot as plt
sys.path.insert(0,'../utils')
from mkdesign import create_design_singlecondition
import scipy.interpolate
from scipy.integrate import odeint
import math
from nipy.modalities.fmri.hemodynamic_models import spm_hrf,compute_regressor



In [ ]:
# first let's build the model without the bilinear influence (aka PPI)
# after http://spm.martinpyka.de/?p=81
nregions=5
z=numpy.zeros(nregions)

# intrinsic connectivity
A=numpy.zeros((z.shape[0],z.shape[0]))
A=numpy.diag(numpy.ones(z.shape[0])*-1)
# add some structure
A=A + numpy.diag(numpy.ones(z.shape[0]-1),k=-1)
B=numpy.zeros(A.shape)
C=numpy.zeros((z.shape[0],1))
C[0]=1
u=0

# we are assuming a 1 second TR for the resulting data
# but the neural data are at a 1/16 millisecond time resolution
stepsize=.01
tslength=300
timepoints=numpy.arange(0,tslength,stepsize)

# create a blocked design
d,design=create_design_singlecondition(blockiness=1.0,deslength=tslength,blocklength=20,offset=20)

u=scipy.interpolate.griddata(numpy.arange(1,d.shape[0]),d,timepoints,fill_value=0)

def find_nearest(array,value):
    idx = numpy.searchsorted(array, value, side="left")
    if math.fabs(value - array[idx-1]) < math.fabs(value - array[idx]):
        return idx-1
    else:
        return idx

def dcm_model(t,z,A,B,C,u,sd):
    ut=numpy.abs(timepoints - t).argmin() 
    return (A.dot(z)+ u[ut]*B.dot(z) + C.dot(u[ut]).T)[0] + numpy.random.rand(len(z))*sd



In [ ]:
noise_mean=1e-9
dcm_model(30,z,A,B,C,u,noise_mean)

In [ ]:
exprate=0.1
zn_noise=zn + numpy.random.exponential(exprate,zn.shape)
hrf=spm_hrf(stepsize,oversampling=1)
zn_noise_conv=numpy.zeros(zn_noise.shape)
for i in range(z.shape[0]):
    zn_noise_conv[:,i]=numpy.convolve(zn_noise[:,i],hrf)[:zn.shape[0]]

In [ ]:
plt.subplot(211)
plt.plot(zn_noise)
plt.subplot(212)
plt.plot(zn_noise_conv)


In [ ]:
def show_graph_with_labels(adjacency_matrix, mylabels):
    import matplotlib.pyplot as plt
    import networkx as nx
    import numpy as np
    rows, cols = np.where(adjacency_matrix == 1)
    edges = zip(rows.tolist(), cols.tolist())
    gr = nx.Graph()
    gr.add_edges_from(edges)
    nx.draw(gr, node_size=500, pos=nx.circular_layout(gr),labels=mylabels, with_labels=True)
    plt.show()

In [ ]:
show_graph_with_labels(A,dict((i,'%d'%i) for i in range(len(z))))

look at a single impulse

In [ ]:
timepoints=numpy.arange(0,20,0.01)
u=numpy.zeros(timepoints.shape[0])
u[10:20]=1

zn_single=odeint(dcm_model,z,timepoints,args=(A,B,C,u,sd))

In [ ]:
plt.plot(timepoints,zn_single)

In [ ]:
plt.plot(d)

In [ ]:
timepoints

In [ ]:
r = scipy.integrate.ode(dcm_model).set_integrator('vode', method='bdf')
r.set_initial_value(z,0)
sd=1e-08
r.set_f_params(A,B,C,u,sd)
r.t=0
data=[]
while r.successful() and r.t <=numpy.max(timepoints):
    try:
        data=numpy.vstack((data,r.integrate(r.t+stepsize)))
    except:
        data=r.integrate(r.t+stepsize)


In [ ]:
plt.plot(data)